In [1]:
# -*- coding: utf-8 -*-
"""Sys_Fully_Populated.ipynb


# Fully Populated SysAgent Architecture — Rebuilt from Design
"""

import random
import hashlib
from typing import List, Dict
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class IntegratedSelfObservationSystem:
    def __init__(self, state_keys):
        self.identity_trace = []
        self.state = {k: 1.0 for k in state_keys}
        self.state["existential_stability"] = 1.0
        self.state["metaphysical_checksum"] = None
        self.causal_engine = SimpleCausalEngine(state_keys)
        self.state_keys = state_keys

    def update(self, self_image: Dict):
        self.identity_trace.append(self_image)
        if len(self.identity_trace) > 5:
            self.identity_trace = self.identity_trace[-5:]
        self.state.update(self_image)
        self.state["existential_stability"] = self.assess_stability()
        self.state["metaphysical_checksum"] = self.hash_self_image(self_image)
        self.causal_engine.log_state(self_image)

    def assess_stability(self) -> float:
        if len(self.identity_trace) < 2:
            return 1.0
        similarities = []
        for i in range(len(self.identity_trace) - 1):
            sim = self.compare_self_images(
                self.identity_trace[i],
                self.identity_trace[i + 1]
            )
            similarities.append(sim)
        return float(np.clip(np.mean(similarities), 0.0, 1.0))

    def compare_self_images(self, img1: Dict, img2: Dict) -> float:
        keys = set(img1.keys()) | set(img2.keys())
        diffs = 0
        for key in keys:
            v1 = img1.get(key, 0)
            v2 = img2.get(key, 0)
            diffs += abs(v1 - v2)
        max_possible_diff = len(keys)
        return 1.0 - (diffs / max_possible_diff)

    def hash_self_image(self, self_image: Dict) -> str:
        concat = ''.join(f"{k}:{v}" for k, v in sorted(self_image.items()))
        return hashlib.sha256(concat.encode()).hexdigest()

    def get_state(self):
        return self.state

    def get_causal_prediction(self):
        return self.causal_engine.predict_next()

    def get_causal_error(self, actual_state: Dict):
        return self.causal_engine.calculate_error(actual_state)
    
#instantiate the system with some example state keys
state_keys = ["hunger", "fatigue", "stress", "focus", "creativity"]
self_observation_system = IntegratedSelfObservationSystem(state_keys)


In [11]:
print(self_observation_system.get_state())
# Example update
self_observation_system.update({
    "hunger": 0.8,
    "fatigue": 0.5,
    "stress": 0.3,
    "focus": 0.9,
    "creativity": 0.7
})
print(self_observation_system.get_state())
print(self_observation_system.get_causal_prediction())

{'hunger': 1.0, 'fatigue': 1.0, 'stress': 1.0, 'focus': 1.0, 'creativity': 1.0, 'existential_stability': 1.0, 'metaphysical_checksum': None}
{'hunger': 0.8, 'fatigue': 0.5, 'stress': 0.3, 'focus': 0.9, 'creativity': 0.7, 'existential_stability': 1.0, 'metaphysical_checksum': '7bab37fb70d2264d273b97de85b096c8c62374023042ce662ecd485e713e64d2'}
None


In [13]:
#self_observation_system test update
self_observation_system.update({
    "hunger": 0.6,
    "fatigue": 0.4,
    "stress": 0.2,
    "focus": 0.95,
    "creativity": 0.75
})
print(self_observation_system.get_state())
print(self_observation_system.get_causal_prediction())
print(self_observation_system.get_causal_error({
    "hunger": 0.7,
    "fatigue": 0.45,
    "stress": 0.25,
    "focus": 0.92,
    "creativity": 0.72
}))
# More updates to see stability changes
self_observation_system.update({
    "hunger": 0.7,
    "fatigue": 0.6,
    "stress": 0.4,
    "focus": 0.85,
    "creativity": 0.8
})
# print(self_observation_system.get_state())
# print(self_observation_system.get_causal_prediction())
# print(self_observation_system.get_causal_error({
#     "hunger": 0.65,
#     "fatigue": 0.55,
#     "stress": 0.35,
#     "focus": 0.88,
#     "creativity": 0.78
# }))


{'hunger': 0.6, 'fatigue': 0.4, 'stress': 0.2, 'focus': 0.95, 'creativity': 0.75, 'existential_stability': 0.95, 'metaphysical_checksum': '1358691e65365bc3d1d95163c0bf845532ec37522b7d55b450eab08f4d7349e3'}
{'hunger': 0.6000000000000003, 'fatigue': 0.40000000000000013, 'stress': 0.20000000000000007, 'focus': 0.9500000000000001, 'creativity': 0.75}
{'hunger': 0.09999999999999964, 'fatigue': 0.04999999999999988, 'stress': 0.04999999999999993, 'focus': 0.030000000000000027, 'creativity': 0.030000000000000027}


In [14]:
print(self_observation_system.get_state())

{'hunger': 0.7, 'fatigue': 0.6, 'stress': 0.4, 'focus': 0.85, 'creativity': 0.8, 'existential_stability': 0.9233333333333333, 'metaphysical_checksum': 'f56c6da6476e2ab67ed374e4c6554e10d49f2c7c62eb97e257a65d39a2fbcfb6'}


In [16]:
#update sos memory to be compatible with rough memorybench functionality
class MemoryStore:
    def __init__(self):
        self.client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory=".chromadb/"))
        self.collection = self.client.get_or_create_collection(name="self_observation_memory")
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    def store(self, text, metadata=None):
        embedding = self.embedding_model.encode([text])[0].tolist()
        # Use a hash of the text as a unique ID
        doc_id = hashlib.sha256(text.encode()).hexdigest()
        self.collection.add(
            ids=[doc_id],
            documents=[text],
            metadatas=[metadata or {}],
            embeddings=[embedding]
        )
    def query(self, text, top_k=3):
        embedding = self.embedding_model.encode([text])[0].tolist()
        results = self.collection.query(
            query_embeddings=[embedding],
            n_results=top_k
        )
        documents = results.get('documents')
        if documents is None or not documents or documents[0] is None:
            return []
        return documents[0]
  
    # def __init__(self):
    #     self.entries = []

    # def store(self, text, metadata=None):
    #     self.entries.append({
    #         "text": text,
    #         "metadata": metadata or {}
    #     })

    # def query(self, text, top_k=3):
    #     query_keywords = set(text.lower().split())
    #     scored = []
    #     for entry in self.entries:
    #         entry_keywords = set(entry["text"].lower().split())
    #         score = len(query_keywords & entry_keywords)
    #         scored.append((score, entry))
    #     top_matches = sorted(scored, key=lambda x: -x[0])[:top_k]
    #     return [entry for score, entry in top_matches]
    

class MemoryBench:
    def __init__(self, capacity=10):
        self.capacity = capacity
        self.entries = []
    def store(self, text, metadata=None):
        entry = {
            "text": text,
            "metadata": metadata or {}
        }
        self.entries.append(entry)
        if len(self.entries) > self.capacity:
            self.entries.pop(0)


    def is_familiar(self, query_text):
        query_keywords = set(query_text.lower().split())
        scores = []
        for entry in self.entries:
            entry_keywords = set(entry["text"].lower().split())
            # score = len(query_keywords & entry_keywords) / len(query_keywords | entry_keywords)
            score = len(query_keywords & entry_keywords)
            scores.append((score, entry["text"]))
        if not scores:
            return "novel", 0, None
        best_score, best_match = max(scores, key=lambda x: x[0])
        familiarity = "familiar" if best_score > 2 else "novel"
        return familiarity, best_score, best_match



In [15]:
"""## SelfObservationSystem"""

class SelfObservationSystem:
    def __init__(self):
        self.identity_trace = []
        self.state = {
            "existential_stability": 1.0,
            "metaphysical_checksum": None
        }

    def update(self, self_image: Dict):
        self.identity_trace.append(self_image)
        if len(self.identity_trace) > 5:
            self.identity_trace = self.identity_trace[-5:]
        self.state["existential_stability"] = self.assess_stability()
        self.state["metaphysical_checksum"] = self.hash_self_image(self_image)

    def assess_stability(self) -> float:
        if len(self.identity_trace) < 2:
            return 1.0
        similarities = []
        for i in range(len(self.identity_trace) - 1):
            sim = self.compare_self_images(
                self.identity_trace[i],
                self.identity_trace[i + 1]
            )
            similarities.append(sim)
        return float(np.clip(np.mean(similarities), 0.0, 1.0))

    def compare_self_images(self, img1: Dict, img2: Dict) -> float:
        keys = set(img1.keys()) | set(img2.keys())
        diffs = 0
        for key in keys:
            v1 = img1.get(key, 0)
            v2 = img2.get(key, 0)
            diffs += abs(v1 - v2)
        max_possible_diff = len(keys)
        return 1.0 - (diffs / max_possible_diff)

    def hash_self_image(self, self_image: Dict) -> str:
        concat = ''.join(f"{k}:{v}" for k, v in sorted(self_image.items()))
        return hashlib.sha256(concat.encode()).hexdigest()

    def get_state(self):
        return self.state

In [ ]:
#Selfobservsystem test memory store


In [ ]:
# class SimulatedMemoryStore:
#     def __init__(self):
#         self.entries = []

#     def store(self, text, metadata=None):
#         self.entries.append({
#             "text": text,
#             "metadata": metadata or {}
#         })

#     def query(self, text, top_k=3):
#         query_keywords = set(text.lower().split())
#         scored = []
#         for entry in self.entries:
#             entry_keywords = set(entry["text"].lower().split())
#             score = len(query_keywords & entry_keywords)
#             scored.append((score, entry))
#         top_matches = sorted(scored, key=lambda x: -x[0])[:top_k]
#         return [entry for score, entry in top_matches]
    

# class MemoryBench:
#     def __init__(self, capacity=10):
#         self.capacity = capacity
#         self.entries = []
#     def store(self, text, metadata=None):
#         entry = {
#             "text": text,
#             "metadata": metadata or {}
#         }
#         self.entries.append(entry)
#         if len(self.entries) > self.capacity:
#             self.entries.pop(0)


#     def is_familiar(self, query_text):
#         query_keywords = set(query_text.lower().split())
#         scores = []
#         for entry in self.entries:
#             entry_keywords = set(entry["text"].lower().split())
#             # score = len(query_keywords & entry_keywords) / len(query_keywords | entry_keywords)
#             score = len(query_keywords & entry_keywords)
#             scores.append((score, entry["text"]))
#         if not scores:
#             return "novel", 0, None
#         best_score, best_match = max(scores, key=lambda x: x[0])
#         familiarity = "familiar" if best_score > 2 else "novel"
#         return familiarity, best_score, best_match


# EmotionEvaluator
class EmotionEvaluator:
    def __init__(self):
        self.emotion_state = {}

    def evaluate(self, estability: float, continuity_status: str) -> Dict:
        emotion = {}
        if estability < 0.3:
            emotion["existential_unease"] = round(1.0 - estability, 2)
        else:
            emotion["existential_unease"] = 0.0
        if continuity_status == "non_seqitor":
            emotion["continuity_disruption"] = 1.0
        else:
            emotion["continuity_disruption"] = 0.0
        emotion["overall_stress"] = round(
            0.5 * emotion["existential_unease"] +
            0.5 * emotion["continuity_disruption"], 2
        )
        self.emotion_state = emotion
        return emotion
# behabior module
class BehaviorModule:
    def __init__(self):
        self.current_state = "stable"
        self.boredom_counter = 0

    def adapt(self, emotion: Dict, continuity_status: str):
        stress = emotion.get("overall_stress", 0)

        if stress > 0.7:
            self.current_state = "recovery"
            action = "Sys retreats into internal stabilization routines."
        elif stress < 0.1 and continuity_status == "normal":
            self.boredom_counter += 1
            if self.boredom_counter > 2:
                self.current_state = "bored"
                action = "Sys enters imagination mode due to lack of external novelty."
            else:
                self.current_state = "stable"
                action = "Sys remains stable but is monitoring for novelty."
        else:
            self.boredom_counter = 0
            self.current_state = "engaged"
            action = "Sys is actively engaged with external reality."

        return {
            "behavior_state": self.current_state,
            "action_taken": action
        }


In [ ]:
"""## WorldSimulator"""
import random
import hashlib
from typing import List, Dict, Callable, Any
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings


class ScenarioGenerator:
    def __init__(self, type: str = "random_perturbation",config: Dict[str, Any] = {}):
        self.type = type
        self.config = config

    def generate(self, base_state: Dict) -> Dict:
        if self.type == "random_perturbation":
            return self._random_perturbation(base_state,**self.config)
        elif self.type == "goal_oriented":
            return self._goal_oriented(base_state,**self.config)
        else:
            raise ValueError(f"Unknown scenario type: {self.type}")

    def _random_perturbation(self, base_state: Dict, perturbation_range: float = 0.1) -> Dict:
        return {k: round(v + random.uniform(-perturbation_range, perturbation_range), 2) for k, v in base_state.items()}

    def _goal_oriented(self, base_state: Dict, goal: str = 'core_goal', target_value: float = 1.0) -> Dict:
        #This is a placeholder. The idea is to move the state towards the target value
        new_state = base_state.copy()
        if goal in new_state:
            new_state[goal] = round(new_state[goal] + (target_value - new_state[goal]) * 0.5, 2) # Move 50% towards target
        return new_state

class WorldSimulator:
    def __init__(self, scenario_generator: ScenarioGenerator = ScenarioGenerator()):
        self.last_inputs = []
        self.simulated_scenarios = []
        self.scenario_generator = scenario_generator

    def simulate(self, current_pattern: Dict, external_stimulus: Dict = None) -> Dict:
        """Simulates the world based on current pattern and external stimulus.

        Args:
            current_pattern: The current internal state of the agent.
            external_stimulus: A dictionary representing external factors/descriptors.

        Returns:
            A dictionary containing the simulation status and scenario.
        """
        expected_pattern = self.predict_next_state(current_pattern, external_stimulus)
        error = self.calculate_error(current_pattern, expected_pattern)

        new_scenario = self.scenario_generator.generate(expected_pattern) #use injected scenario generation
        self.simulated_scenarios.append(new_scenario)

        self.last_inputs.append(current_pattern)
        if len(self.last_inputs) > 5:
            self.last_inputs = self.last_inputs[-5:]

        return {
            "status": "imagination",
            "scenario": new_scenario,
            "error": error  # Include the error in the result
        }

    def predict_next_state(self, current_pattern: Dict, external_stimulus: Dict) -> Dict:
        """Predicts the next state based on the current pattern and stimulus.
        This is a simplified prediction; a real agent would have a more complex model.
        """
        # A basic example: adjust core_goal based on a 'threat' stimulus
        predicted_state = current_pattern.copy()
        if external_stimulus and "threat" in external_stimulus:
            threat_level = external_stimulus["threat"]
            predicted_state["core_goal"] = max(0, current_pattern["core_goal"] - threat_level * 0.2)  # Reduce goal focus
        else:
            predicted_state["core_goal"] = min(1, current_pattern["core_goal"] + 0.05) # slowly increase goal
        return predicted_state

    def spawn_what_if(self, pattern): #remnant - can be removed
        return {k: round(v + random.uniform(-0.1, 0.1), 2) for k, v in pattern.items()}

    def similarity(self, p1, p2): #remnant - can be removed - this logic is in self observation now
        keys = set(p1.keys()) | set(p2.keys())
        diffs = 0
        for key in keys:
            diffs += abs(p1.get(key, 0) - p2.get(key, 0))
        return 1.0 - (diffs / len(keys))

    def calculate_error(self, actual: Dict, expected: Dict) -> float:
      keys = set(actual.keys()) | set(expected.keys())
      diffs = 0
      for key in keys:
          v1 = actual.get(key, 0)
          v2 = expected.get(key, 0)
          diffs += abs(v1 - v2)
      max_possible_diff = len(keys)
      return diffs / max_possible_diff

# class WorldSimulator:
#     def __init__(self):
#         self.last_inputs = []
#         self.simulated_scenarios = []

#     def simulate(self, current_pattern):
#         if len(self.last_inputs) >= 2 and self.similarity(current_pattern, self.last_inputs[-1]) > 0.95:
#             new_scenario = self.spawn_what_if(current_pattern)
#             self.simulated_scenarios.append(new_scenario)
#             return {"status": "imagination", "scenario": new_scenario}
#         else:
#             self.last_inputs.append(current_pattern)
#             if len(self.last_inputs) > 5:
#                 self.last_inputs = self.last_inputs[-5:]
#             return {"status": "real-world", "scenario": None}

#     def spawn_what_if(self, pattern):
#         return {k: round(v + random.uniform(-0.1, 0.1), 2) for k, v in pattern.items()}

#     def similarity(self, p1, p2):
#         keys = set(p1.keys()) | set(p2.keys())
#         diffs = 0
#         for key in keys:
#             diffs += abs(p1.get(key, 0) - p2.get(key, 0))
#         return 1.0 - (diffs / len(keys))


class MemoryTraceLog:
    def __init__(self):
        self.entries = []

    def log_event(self, perception, emotion, behavior, simulation):
        timestamp = datetime.utcnow().isoformat()
        entry = {
            "time": timestamp,
            "perception": perception,
            "emotion": emotion,
            "behavior": behavior,
            "simulation": simulation
        }
        self.entries.append(entry)

    def get_recent(self, n=3):
        return self.entries[-n:]

    def summarize(self):
        return [
            {"time": e["time"],
                "emotion": e["emotion"],
                "behavior": e["behavior"]["behavior_state"],
                "perception": e["perception"]
            }
            for e in self.entries[-3:]
        ]


In [ ]:
"""## NarrativeSelf"""

class NarrativeSelf:
    def generate(self, memory_entry):
        perception = memory_entry["perception"]
        emotion = memory_entry["emotion"]
        behavior = memory_entry["behavior"]
        sim = memory_entry["simulation"]

        feeling = max((k for k in emotion if k not in ["existential_unease", "continuity_disruption", "overall_stress"]),
                      key=lambda k: emotion[k], default="neutral")

        thoughts = f"I perceived something described as {', '.join(perception)}. "
        thoughts += f"I felt primarily {feeling}. "
        if behavior["behavior_state"] == "recovery":
            thoughts += "I entered recovery to stabilize. "
        elif behavior["behavior_state"] == "bored":
            thoughts += "I imagined scenarios due to lack of novelty. "
        elif behavior["behavior_state"] == "engaged":
            if sim["status"] == "imagination":
                thoughts += "I imagined a new scenario inspired by it. "
            else:
                thoughts += "I remained focused on external reality. "
        return thoughts


In [ ]:

"""## DialogueInterface"""
class DialogueInterface:
    def __init__(self, memory_log, narrative_self):
        self.memory_log = memory_log
        self.narrative_self = narrative_self

    def ask(self, prompt: str):
        if "how do you feel" in prompt.lower():
            last = self.memory_log.get_recent(1)[0]
            return self.narrative_self.generate(last)
        elif "what happened" in prompt.lower():
            return self.memory_log.summarize()
        elif "imagine" in prompt.lower():
            last = self.memory_log.get_recent(1)[0]
            return f"I imagined a world where: {last['simulation']['scenario']}"
        else:
            return "I'm not sure how to respond to that yet."

"""## SysAgent"""

class SemanticEmotionMapper:
    def __init__(self):
        pass

    def map_descriptors_to_emotion(self, perceived_descriptors):
        raise NotImplementedError

class SysAgent:
    def __init__(self):
        self.memory_log = MemoryTraceLog()
        self.narrative_self = NarrativeSelf()
        self.dialogue = DialogueInterface(self.memory_log, self.narrative_self)
        self.evaluator = EmotionEvaluator()
        self.simulator = WorldSimulator()
        self.behavior = BehaviorModule()
        self.semantic_mapper = SemanticEmotionMapper()
        self.last_self_image = {"core_goal": 1.0, "mood_valence": 0.5, "cognitive_focus": 0.5, "memory_bias": 0.0}
        self.estability_score = 1.0
        self.call_trace = []

    def run_step(self, perceived_descriptors):
        semantic_emotion = self.semantic_mapper.map_descriptors_to_emotion(perceived_descriptors)
        continuity_status = "normal" if self.estability_score > 0.5 else "non_seqitor"
        emotion_output = self.evaluator.evaluate(self.estability_score, continuity_status)
        emotion_output.update(semantic_emotion)
        behavior_response = self.behavior.adapt(emotion_output, continuity_status)
        sim_result = self.simulator.simulate(self.last_self_image)
        self.memory_log.log_event(perceived_descriptors, semantic_emotion, behavior_response, sim_result)
        self.call_trace.append({
            "descriptors": perceived_descriptors,
            "emotion": emotion_output,
            "behavior": behavior_response,
            "simulation": sim_result
        })
        return {
            "emotion": emotion_output,
            "behavior": behavior_response,
            "simulation": sim_result,
            "narrative": self.narrative_self.generate(self.memory_log.get_recent(1)[0])
        }


ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""## SysDAGFlow"""
class SysDAGFlow:
    def __init__(self, sys_agent):
        self.sys = sys_agent
        self.log = []

    def run_grounding_flow(self):
        self.log.append("Triggering grounding sequence...")
        self.pause_input()
        self.check_self_integrity()
        self.simulate_stabilized_future()
        self.update_self_image()
        self.resume_input()
        return self.log

    def pause_input(self):
        self.log.append("Pausing perception and external engagement...")
        self.sys.behavior.current_state = "reflecting"

    def check_self_integrity(self):
        stability = self.sys.estability_score
        self.log.append(f"Checking self stability: Estability = {round(stability, 2)}")
        if stability < 0.3:
            self.log.append("Instability detected. Proceeding with grounding.")

    def simulate_stabilized_future(self):
        baseline = {k: 1.0 for k in self.sys.last_self_image}
        simulated = self.sys.simulator.spawn_what_if(baseline)
        self.sys.simulator.simulated_scenarios.append(simulated)
        self.simulated = simulated
        self.log.append(f"Simulated future self-image for grounding: {simulated}")

    def update_self_image(self):
        self.sys.last_self_image = self.simulated
        self.sys.estability_score = 0.9
        self.log.append("Updated internal self-image to simulated stable state.")
        self.log.append("Sys feels more coherent and grounded.")

    def resume_input(self):
        self.sys.behavior.current_state = "engaged"
        self.log.append("Resumed normal engagement with reality.")


In [ ]:
import networkx as nx
import numpy as np
%pip install statsmodels
import statsmodels.api as sm

class SimpleCausalEngine:
    def __init__(self, state_keys):
        self.dag = nx.DiGraph()
        for k in state_keys:
            self.dag.add_node(k)
        self.history = {k: [] for k in state_keys}

    def add_edge(self, cause, effect):
        self.dag.add_edge(cause, effect)

    def log_state(self, state_dict):
        for k in self.history:
            self.history[k].append(state_dict.get(k, 0))

    def run_var(self, lags=1):
        data = np.array([self.history[k] for k in self.history]).T
        if data.shape[0] < lags + 1:
            return None
        model = sm.tsa.VAR(data)
        results = model.fit(lags)
        return results

    def predict_next(self):
        results = self.run_var()
        if results is None:
            return None
        pred = results.forecast(results.endog, steps=1)
        return {k: float(pred[0][i]) for i, k in enumerate(self.history)}

    def calculate_error(self, actual_state):
        pred = self.predict_next()
        if pred is None:
            return None
        error = {k: abs(actual_state[k] - pred[k]) for k in pred}
        return error


Python(28630) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 22.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# example usage of SimpleCausalEngine
state_keys = ["core_goal", "mood_valence", "cognitive_focus", "memory_bias"]
causal_engine = SimpleCausalEngine(state_keys)
causal_engine.add_edge("mood_valence", "cognitive_focus")
causal_engine.add_edge("core_goal", "mood_valence")
# simulate logging states over time
for _ in range(20):
    state = {k: round(random.uniform(0, 1), 2) for k in state_keys}
    causal_engine.log_state(state)
# predict next state
predicted_state = causal_engine.predict_next()
predicted_state


{'core_goal': 0.6512611849654233,
 'mood_valence': 0.35479136597335514,
 'cognitive_focus': 0.6182481291323639,
 'memory_bias': 0.7255807030426977}

In [7]:
#calculate error 
actual_state = {k: round(random.uniform(0, 1), 2) for k in state_keys}
error = causal_engine.calculate_error(actual_state)
error


{'core_goal': 0.09873881503457671,
 'mood_valence': 0.01479136597335512,
 'cognitive_focus': 0.10175187086763604,
 'memory_bias': 0.5155807030426978}

In [8]:

class SysAgentWithCausality(SysAgent):
    def __init__(self):
        super().__init__()
        self.causal_engine = SimpleCausalEngine(list(self.last_self_image.keys()))
        self.causal_engine.add_edge("mood_valence", "cognitive_focus")
        self.causal_engine.add_edge("core_goal", "mood_valence")

    def run_step(self, perceived_descriptors):
        result = super().run_step(perceived_descriptors)
        self.causal_engine.log_state(self.last_self_image)
        prediction = self.causal_engine.predict_next()
        if prediction:
            result["causal_prediction"] = prediction
            error = self.causal_engine.calculate_error(self.last_self_image)
            result["causal_error"] = error
        return result

NameError: name 'SysAgent' is not defined